 # Day 2 Exercise 3 - image segmentation and tracking: the classic way
 ## Basic Definitions
 ### Superpixels
 When we group pixels based on color, texture, or other low level primitives, we call these perceptual groups superpixels
 ![superpixel](Data/superpixel.gif)

 ### Semantic Segmentation
 In Semantic Segmentation the goal is to assign a label (car, building, person, road, sidewalk, sky, trees etc.) to __every pixel__ in the image.
 ![semantic](Data/semantic.jpg)

 ### Instance Segmentation
 Instance Segmentation is a concept closely related to Object Detection. However, unlike Object Detection the output is a mask (or contour) containing the object instead of a bounding box. Unlike Semantic Segmentation, we do not label every pixel in the image; we are interested only in finding the boundaries of specific objects
 ![instance](Data/instance.jpg)

 Panoptic segmentation is the combination of Semantic segmentation and Instance Segmentation. Every pixel is assigned a class (e.g. person), but if there are multiple instances of a class, we know which pixel belongs to which instance of the class.
 ![panoptic](Data/panoptic.jpg)

 ## Example: Classic Segmentation with `scikit-image`

 ### Basic `scikit-image` i/o
 Importing a GrayScale Image from the skimage library

In [0]:
from skimage import data
import numpy as np
import matplotlib.pyplot as plt
image = data.binary_blobs()
plt.imshow(image, cmap='gray')

 Importing a Colored Image from the skimage library

In [0]:
from skimage import data
import numpy as np
import matplotlib.pyplot as plt
image = data.astronaut()
plt.imshow(image)

 Importing an image from an external source

In [0]:
# The I/O module is used for importing the image
from skimage import data
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
image = io.imread('skimage_logo.png')
plt.imshow(image)


 ### Segmentation: preparation


In [0]:
# basic imports
import numpy as np
import matplotlib.pyplot as plt
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
# A simple function to plot the images
def image_show(image, nrows=1, ncols=1, cmap='gray'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 14))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

 ### Superivised Thresholding
 read the image we want to segment

In [0]:
text = data.page()
image_show(text)

For this segmentation task, a manual method would be to accept pixels darker than a given value (threshold) as objects, and the rest as background. Now to help us in picking that value, we will use a Histogram.

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(text.ravel(), bins=32, range=[0, 256])
ax.set_xlim(0, 256);

In [ ]:
text_segmented = text > 50 # (value concluded from histogram i.e 50,70,120 )
image_show(text_segmented)

### Unsupervised Thresholding


In [ ]:
text_threshold = filters.threshold_  # Hit tab with the cursor after the underscore to get all the methods.
image_show(text < text_threshold);

### Supervised Segmentation

Thresholding is a very basic segmentation process and will not work properly in a high-contrast image for which we will be needing more advanced tools.

In [ ]:
# import the image
from skimage import io
image = io.imread('girl.jpg') 
plt.imshow(image);

In [ ]:
#Before doing any segmentation on an image, it is a good idea to de-noise it using some filters.
image_gray = color.rgb2gray(image) 
image_show(image_gray);

### Supervised Segmentation: Active contour segmentation

Active Contour segmentation is initialized using a user-defined contour or line, around the area of interest and this contour then slowly contracts and is attracted or repelled from light and edges.

In [ ]:
# draw a circle around the person’s head to initialize the snake.
def circle_points(resolution, center, radius):
    """
    Generate points which define a circle on an image.Centre refers to the centre of the circle
    """   
    radians = np.linspace(0, 2*np.pi, resolution)
    c = center[1] + radius*np.cos(radians)#polar co-ordinates
    r = center[0] + radius*np.sin(radians)
    
    return np.array([c, r]).T

# Exclude last point because a closed path should not have duplicate points
points = circle_points(200, [80, 250], 80)[:-1]

fig, ax = image_show(image)
ax.plot(points[:, 0], points[:, 1], '--r', lw=3)

snake = seg.active_contour(image_gray, points)
fig, ax = image_show(image)
ax.plot(points[:, 0], points[:, 1], '--r', lw=3)
ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3);

# tweak the parameters called alpha and beta. Higher values of alpha will make this snake contract faster while beta makes the snake smoother.

snake = seg.active_contour(image_gray, points,alpha=0.06,beta=0.3)
fig, ax = image_show(image)
ax.plot(points[:, 0], points[:, 1], '--r', lw=3)
ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3);

### Supervised Segmentation: random walker segmentation

Further reading: [this link](https://ieeexplore.ieee.org/document/1704833?source=post_page).

First we label part of the pixels in the image.

In [ ]:
image_labels = np.zeros(image_gray.shape, dtype=np.uint8)

indices = draw.circle_perimeter(80, 250,20)#from here
image_labels[indices] = 1
image_labels[points[:, 1].astype(np.int), points[:, 0].astype(np.int)] = 2
image_show(image_labels);

In [ ]:
# use Random Walker and see what happens.
image_segmented = seg.random_walker(image_gray, image_labels)
# Check our results
fig, ax = image_show(image_gray)
ax.imshow(image_segmented == 1, alpha=0.3);

We can tune in the beta parameter until we get the desired results. After several attempts, a value of 3000 works reasonably well.

In [ ]:
image_segmented = seg.random_walker(image_gray, image_labels, beta = 3000)
# Check our results
fig, ax = image_show(image_gray)
ax.imshow(image_segmented == 1, alpha=0.3);

### Unsupervised segmentation: SLIC( Simple Linear Iterative Clustering)

It uses machine-learning methods under the hood, but users do not necessarily realize that. More information can be found [here](https://ieeexplore.ieee.org/document/6205760?source=post_page).

In [ ]:
image_slic = seg.slic(image,n_segments=155)

# label2rgb replaces each discrete label with the average interior color
image_show(color.label2rgb(image_slic, image, kind='avg'));


### Unsupervised segmentation: Felzenszwalb

This algorithm also uses a machine learning algorithm called minimum-spanning tree clustering under the hood. The Reference Paper can be accessed [here](http://cs.brown.edu/people/pfelzens/segment/?source=post_page).

In [ ]:
image_felzenszwalb = seg.felzenszwalb(image) 
image_show(image_felzenszwalb);

These are a lot of regions. Let’s calculate the number of unique regions and recolour them using the region average just as we did in the SLIC algorithm. 

In [ ]:
image_felzenszwalb_colored = color.label2rgb(image_felzenszwalb, image, kind='avg')
image_show(image_felzenszwalb_colored)